<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #6: Word2Vec

`Fecha de entrega: Septiembre 26, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

### Estudiante: Ilvar Dario Sanabria G. 
### Código      : 616201058

In [1]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords #Palabras vacias
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer #Stemmer
spanishStemmer=SnowballStemmer("spanish")


# Punto 1: Pre-Procesamiento

- `[18 pts]` Leer el archivo `Princesas.csv` usando `pandas` y crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [49]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto.split()

### Busque un texto sobre planificación familiar, lo econtre en PDF y lo converti a Texto

In [83]:
textoPF = pd.read_fwf("c:/tmp/PF texto02.txt", encoding='utf-8',sep=" ",header=None )
textoPF

,0
0,SPINE
1,Planificación familiar
2,Planificación familiar
3,2019
4,EDICIÓN DEL 2019
...,...
23953,Menos de 1 embarazo por
23954,cada 100 mujeres en un año
23955,Más efectivo
23956,NUEVO


In [84]:
textoPF['pp'] = textoPF[0].apply(lambda texto: pre_procesado(texto))

textoPF.head()

,0,pp
0,SPINE,[spine]
1,Planificación familiar,"[planificación, familiar]"
2,Planificación familiar,"[planificación, familiar]"
3,2019,[]
4,EDICIÓN DEL 2019,[edición]


In [76]:
import gensim.models.word2vec as w2v

In [85]:
textoPF['pp'].values

array([list(['spine']), list(['planificación', 'familiar']),
       list(['planificación', 'familiar']), ..., list(['efectivo']),
       list(['nuevo']), list([])], dtype=object)

### Se genera un modelo con base en el texto de PF

In [86]:
mi_modelo = w2v.Word2Vec(textoPF['pp'].values,
                          sg=1, # 1 skip-gram. 0 CBOW
                          seed=24092020, # semilla
                          size=256, # número de dimensiones
                          min_count=50,
                          window=12)

In [101]:
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

### Buscamos parabras similares para algunos terminos tipicos de la jerga de Planificación Familiar

In [88]:
#obtener palabras similares
vocab = [e[0] for e in mi_modelo.wv.most_similar("hijos")]
print(vocab)


['hijo', 'además', 'ejemplo', 'período', 'usa', 'hombre', 'pene', 'm', 'ambos', 'diafragma']


In [89]:
vocab = [e[0] for e in mi_modelo.wv.most_similar("mujer")]
print(vocab)

['quiere', 'menstruación', 'restablecido', 'si', 'seguir', 'incluso', 'siempre', 'afección', 'vez', 'algún']


In [90]:
vocab = [e[0] for e in mi_modelo.wv.most_similar("pareja")]
print(vocab)

['cómo', 'personas', 'parejas', 'hombre', 'hijos', 'usuario', 'mayor', 'tener', 'usuaria', 'usan']


In [96]:
vocab = [e[0] for e in mi_modelo.wv.most_similar("embarazo")]
print(vocab)

['síntomas', 'haga', 'afecciones', 'seguimiento', 's', 'prevención', 'pélvica', 'l', 'tratamiento', 'mayor']


In [113]:
vocab = [e[0] for e in mi_modelo.wv.most_similar("sexo")]
print(vocab)


KeyError: "word 'sexo' not in vocabulary"

In [108]:
#Se obtiene la matriz
X = mi_modelo[mi_modelo.wv.vocab]
matrix = pd.DataFrame(X)
matrix.index = mi_modelo.wv.vocab.keys()
matrix

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
planificación,0.484532,0.031767,0.180959,-0.300420,0.229313,0.165234,0.289575,-0.139772,0.033905,0.188920,...,-0.094749,-0.125020,-0.085986,-0.164748,0.467016,0.155249,-0.103843,0.221451,-0.150140,-0.070342
familiar,0.451668,0.030278,0.170718,-0.275606,0.197108,0.149303,0.292427,-0.173422,0.032559,0.205342,...,-0.086525,-0.138698,-0.108876,-0.187925,0.505206,0.178896,-0.108134,0.237065,-0.166672,-0.070168
embarazos,0.190497,-0.032928,-0.026884,-0.106884,0.152387,0.016502,0.153093,0.010658,0.057413,-0.008811,...,-0.072826,-0.010502,-0.058083,-0.020084,0.122712,0.044625,-0.057293,0.115261,0.087659,-0.001141
cada,0.152270,-0.032350,-0.040603,-0.080021,0.127565,0.004792,0.149220,-0.003396,0.065097,-0.013324,...,-0.079053,0.003584,-0.076410,-0.019393,0.118536,0.060567,-0.064124,0.113948,0.109067,0.018467
mujeres,0.116867,-0.037226,-0.022280,-0.044902,0.083390,-0.027420,0.113128,-0.019506,0.034965,-0.007491,...,-0.057785,-0.003837,-0.066075,-0.018531,0.168266,0.044770,-0.015982,0.103789,0.105032,0.015013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
útero,0.178249,-0.027496,-0.028352,-0.096632,0.160710,-0.014247,0.154349,0.025201,0.033649,-0.033970,...,-0.056808,-0.015325,-0.029319,0.001026,0.091136,0.033957,-0.042666,0.099330,0.075589,-0.020234
zona,0.191682,-0.030788,-0.025539,-0.118433,0.172381,-0.006820,0.160489,0.026208,0.044212,-0.027648,...,-0.059722,-0.016316,-0.024004,-0.001848,0.092724,0.048876,-0.038121,0.102519,0.065685,-0.031930
capuchón,0.194403,-0.033792,-0.019391,-0.118012,0.172176,0.006997,0.157148,0.036829,0.046902,-0.019826,...,-0.061665,-0.024074,-0.032483,-0.008971,0.100153,0.034106,-0.050410,0.104034,0.072846,-0.020959
pene,0.165317,-0.033421,-0.021785,-0.089898,0.144358,-0.020891,0.148253,0.013346,0.040049,-0.021151,...,-0.053575,-0.016447,-0.038927,-0.006977,0.102870,0.053387,-0.040216,0.104648,0.067675,-0.015055


In [105]:
#Se cambia el tamaño a dos componentes
pca = PCA(n_components=2)
 
result = pca.fit_transform(matrix)
result = pd.DataFrame(result)
result.columns = ['X', 'Y']
result['Palabra'] = matrix.index.values
 
result

,X,Y,Palabra
0,1.759023,2.062819,planificación
1,1.733612,2.210212,familiar
2,0.044063,0.039438,embarazos
3,-0.159692,-0.027642,cada
4,-0.326172,0.050669,mujeres
...,...,...,...
301,-0.025136,-0.121583,útero
302,0.055360,-0.087240,zona
303,0.083582,-0.042596,capuchón
304,-0.049722,-0.034791,pene


In [106]:
#Se grafica la distacia de palabras
trace = go.Scatter(x=result['X'].values,
                    y=result['Y'].values,
                    text=result['Palabra'].values,
                    mode='markers') 
 
layout = go.Layout(title="PCA") 
fig = go.Figure(data=trace, layout=layout)

iplot(fig)

Se observa que las palabras estan bastante concentradas en la grafica y las que son outliers , son palabras que no tienen ningun significado. se debe revisar el preprocesado del texto 

In [ ]:
### TRatamos de usar la funcion de accuracy para medir la efectividad del modelo, se creo un cnutno de palabras de evaluación

In [114]:
efectividad = mi_modelo.wv.accuracy('c:/tmp/evalPF.txt')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.evaluate_word_analogies() instead).



In [115]:
print(efectividad)

[{'section': 'palabras-comunes-planificacion', 'correct': [], 'incorrect': []}, {'section': 'total', 'correct': [], 'incorrect': []}]


### ¿Su modelo da buenos resultados? ¿Por qué sí o por qué no?

Entiendo que la fortaleza del modelo se basa en el corpus que se usa para alimentarlo, en el ejemplo qu yo tome solo se tomo un texto sobre planificación Familiar, por lo tanto los resultados pueden no ser tan buenos. Pues buscando palabras similares que tien que ver sobre esta tematica y que son bastante frecuentes, no se econtraron resultados, ej sexo.
El investigar y obtener corpus robustos, con seguridad mejora el desempeño del modelo


### ¿Qué problemas encontró al realizar este taller?

1- Encontrar corpus sobre la tematica seleccionada : PLanificación Familiar
2- Entender la documentacion , no es facil comprender sin ejemplos
3- En consecuencia con el punto anterior, no me fue posible medir la efectivad del modelo pues a pesar de construir un texto de evaluación no entendi los resultados